In [2]:
import pickle
import numpy as np
import os
from music21 import instrument, note, stream, chord

from fractions import Fraction

import parsing_midi
import models

Using TensorFlow backend.
/home/ozmitche/anaconda3/envs/tensorflow_cpu_old/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ozmitche/anaconda3/envs/tensorflow_cpu_old/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ozmitche/anaconda3/envs/tensorflow_cpu_old/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qi

In [13]:
#the file of learned weights we wish to use
weight_file = 'input_batch_weights/olivia_run_0weights-improvement-14-6.3849-bigger.hdf5'
#length of output to generate
output_lenght = 500
#length of input sequence of notes to generate the next note
#make sure this matches the model the weights were trained on!
sequence_length = 100
#notesfile, has a list of all the different notes in the training set
#this needs to match the model the weights were trained on
notefile = 'notes_olivia_quarter_length_batch'
#file to generate harmony with
song = 'ff7-mainmidi'
#offset of generated notes in file
_offset = 0.5

In [14]:
#This section runs the model once the weights are generated
#generates an output (numerical output) that continues the input melody
def generate_notes(model, network_input, pitchnames, n_vocab):
    #use a random input set as the seed
    start = np.random.randint(0, len(network_input)-1)  
    
    #pattern is the initial melody seed we will input
    pattern = network_input[start]
    
    prediction_output = []
    
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    # generate some number of notes
    for note_index in range(output_lenght):
        #aligning input pattern with model input size
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        
        #predict the next note
        prediction = model.predict(prediction_input, verbose=0)
        #end result of neural network is a list of probabilities this is the next note
        #grab note with highest probability
        index = np.argmax(prediction)
        result = int_to_note[index]
        
        #add predicted note to our output
        prediction_output.append(result)
        
        #add predicted note to end of input pattern and cut off begining of pattern
        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]
    
    return prediction_output

In [15]:
#This section decodes the RNN friendly note format
def decode (prediction_output):
    offset = 0
    output_notes = []
    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        pattern_note = pattern.split('-')
        pattern = pattern_note[0]
        length = int(Fraction(pattern_note[1]))
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            new_chord.quarterLength = length
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            new_note.quarterLength = length
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += 0.5
        
    return output_notes

In [16]:
#This section creates the midi file
def make_midi(output_notes):
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='input_batch_output_melody_w6.3849(1).mid')

In [17]:
#Main function to cover using the model to generate a midi file
def generate():
    #load the notes used to train the model
    if os.path.isfile('data/'+notefile):
        with open('data/'+notefile, 'rb') as filepath:
            notes = pickle.load(filepath)
            print("load notes from file")
    else:
        notes = parsing_midi.get_notes()
        print("load notes from function")
    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = parsing_midi.prepare_sequence_in(notes, n_vocab, sequence_length)
    
    model = models.create_lstm_network_with_batch(normalized_input, n_vocab)
    model.load_weights(weight_file)
    
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    make_midi(decode(prediction_output))

In [18]:
#if called from commandline, this should just try rnning the network
#if called in a notebook, will just run
#if __name__ == '__main__':
    generate()

load notes from file


In [41]:
#This section runs the model once the weights are generated
#generates an output (numerical output) that complements the input melody
def generate_harmony(model, song_input, pitchnames, n_vocab):
    
    #pattern is the  melody seed we will input
    #This is the first batch of notes in the melody
    pattern = song_input[0:sequence_length]
    
    #Since this generates the note that comes next
    #the harmony actually start at note <sequence_length> in the melody
    #putting notes from the original pattern here as placeholder
    prediction_output = []
    
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    # generate some number of notes
    for note_index in range(len(song_input)-sequence_length):
        #aligning input pattern with model input size
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        
        #predict the next note
        prediction = model.predict(prediction_input, verbose=0)
        #end result of neural network is a list of probabilities this is the next note
        #grab note with highest probability
        index = np.argmax(prediction)
        result = int_to_note[index]
        
        #add predicted note to our output
        prediction_output.append(result)
        
        #add predicted note to end of input pattern and cut off begining of pattern
        pattern = np.append(pattern, song_input[note_index+sequence_length])
        pattern = np.append(pattern, index)
        pattern = pattern[2:len(pattern)]

    return prediction_output

In [54]:
#This section creates the midi file
def make_midi_h(output_notes, output_m_notes):
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_output.mid')
    midi_stream = stream.Stream(output_m_notes)
    midi_stream.write('midi', fp='test_output_m.mid')

In [55]:
#This section decodes the RNN friendly note format
def decode_harmony (prediction_output, song_input, pitchnames):
    offset = 0
    output_notes = []
    output_m_notes = []
    
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    # create note and chord objects based on the values generated by the model
    input_notes = []
    for idx in range(len(song_input)):
        m_note = int_to_note[song_input[idx]]
        input_notes = np.append(input_notes, m_note)
    
    for pattern in input_notes:
        pattern_note = pattern.split('-')
        pattern = pattern_note[0]
        length = int(Fraction(pattern_note[1]))
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            new_chord.quarterLength = length
            output_m_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            new_note.quarterLength = length
            output_m_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += _offset

    offset = _offset*(sequence_length)

    for pattern in prediction_output:
        pattern_note = pattern.split('-')
        pattern = pattern_note[0]
        length = int(Fraction(pattern_note[1]))
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            new_chord.quarterLength = length
            output_notes.append(new_chord)
            output_m_notes.append(new_note)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            new_note.quarterLength = length
            output_notes.append(new_note)
            output_m_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += _offset
        
    return (output_notes, output_m_notes)

In [56]:
#Main function to cover using the model to generate a midi file
def generate_h():
    #load the notes used to train the model
    if os.path.isfile('data/'+notefile):
        with open('data/'+notefile, 'rb') as filepath:
            notes = pickle.load(filepath)
            print("load notes from file")
    else:
        notes = parsing_midi.get_notes()
        print("load notes from function")
    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = parsing_midi.prepare_sequence_in(notes, n_vocab, sequence_length)
    song_input = parsing_midi.prepare_melody_in(notes, n_vocab, song)
    
    model = models.create_lstm_network_with_batch(normalized_input, n_vocab)
    model.load_weights(weight_file)
    
    prediction_output= generate_harmony(model, song_input, pitchnames, n_vocab)
    output_notes, output_m_notes = decode_harmony(prediction_output, song_input, pitchnames)
    make_midi_h(output_notes, output_m_notes)

In [58]:
#if called from commandline, this should just try rnning the network
#if called in a notebook, will just run
#if __name__ == '__main__':
    generate_h()

load notes from file
